In [2]:
# This file is an implementation of the 2D LIPM
# Author : Avadesh Meduri
# Date : 21/04/2020

import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

In [3]:
'''
This is an implementation of a 2D Lipm environment with variable number of steps possible
'''

class TwoDLipmEnv:
    
    def __init__(self, h, b, max_step_length, w, no_actions = [11, 9]):
        '''
        Input:
            h : height of the lipm above the ground
            b : width of the base (distance between the feet)
            max_step_length : max step length allowed
            w : weights for the cost computation
            no_actions : number of discretizations
        '''
        
        self.omega = np.sqrt(9.81/h)
        self.max_leg_length = 0.3
        self.dt = 0.001
        self.h = h
        self.b = b
        self.no_steps = 0
        assert len(w) == 3
        self.w = w
        assert (np.linalg.norm([max_step_length, self.h]) < self.max_leg_length)
        assert len(no_actions) == 2
        # The co ordinate axis is x : forward and y : sideways walking, z : faces upward
        # This means that left leg is on the positive side of the y axis
        # The addition b is added to accomodate a step length larger than leg length as it may be feasible
        # in high velocity cases.
        self.action_space_x = np.around(np.linspace(-max_step_length, max_step_length, no_actions[0]), 2)
        # actions to the free side
        if b > 0 :
            self.action_space_ly = np.geomspace(b, max_step_length + b, int(2*no_actions[1]/3))
            # actions to the non free side where leg can hit the other leg
            # Y axis actions step length allowed such that robot can't step to the left of the left leg
            # or the right to the right leg (no criss crossing)
#             self.action_space_ry = np.arange(0, b, self.action_space_ly[1] - self.action_space_ly[0])
            self.action_space_ry = np.linspace(0, b, int(no_actions[1]/3), endpoint = False)
            self.action_space_y = np.around(np.concatenate((self.action_space_ry, self.action_space_ly)), 2)
        else:
            self.action_space_y = np.around(np.linspace(0, max_step_length, int(no_actions[1]/2)), 2)
            
        self.A = np.matrix([[1, 0, self.dt, 0], 
                            [0, 1, 0, self.dt], 
                            [self.dt*(self.omega**2), 0, 1, 0], 
                            [0, self.dt*(self.omega**2), 0, 1]])
        
        self.B = np.matrix([[0, 0], [0, 0], [-(self.omega**2)*self.dt, 0], [0, -(self.omega**2)*self.dt]])
        self.t = 0
        
    def integrate_lip_dynamics(self, x_t, u_t):
        '''
        integrates the dynamics of the lipm for one time step
        Input:
            x_t : current state of the lip ([x_com, y_com, xd_com, yd_com])
            u_t : current cop location ([u_x, u_y])
        '''
        assert np.shape(x_t) == (4,)
        x_t_1 = np.matmul(self.A, np.transpose(x_t)) + np.matmul(self.B, np.transpose(u_t))
        return x_t_1
    
    def reset_env(self, x0, v_des, epi_time):
        '''
        Resets environment for a new episode
        Input:
            x0 : initial state of the system [x, y, xd, yd]
            v_des : desired velocity [xd_des, yd_des]
            epi_time : episode time
        '''
        assert np.shape(x0) == (4,)
        self.t = 0
        # [x, y, xd, yd, ux, uy, h, n]
        self.sim_data = np.zeros((8, int(epi_time/self.dt)+1))
        self.no_steps = 0
        assert (len(v_des) == 2)
        self.v_des = v_des
        assert (np.linalg.norm([x0[0], self.h]) < self.max_leg_length)
        assert (np.linalg.norm([x0[1], self.h]) < self.max_leg_length)
        self.sim_data[:,0][0:4] = x0
        self.sim_data[:,0][5] = -self.b/2 # right leg on the ground
        self.sim_data[:,0][6] = self.h
        self.sim_data[:,0][7] = 1 # determines which leg is on the ground (1 is right leg)
        
        processed_state = np.zeros(7)
        processed_state[0:5] = np.take(self.sim_data[:,0], [0, 1, 2, 3, 7])
        processed_state[5:7] = self.v_des    
        
        return processed_state
    
    def step_env(self, u, step_time):
        '''
        Integrates the dynamics of the lipm for the duration of a step (until next action is to be taken)
        Input:
            u : action (next step)
            step_time : the duration after which next step is taken
        '''
        for i in range(int(step_time/self.dt)):
            self.sim_data[:,self.t + 1][0:4] = self.integrate_lip_dynamics(self.sim_data[:,self.t][0:4], \
                                                                           self.sim_data[:,self.t][4:6])
            self.sim_data[:,self.t + 1][4:6] = self.sim_data[:,self.t][4:6] # u
            self.sim_data[:,self.t + 1][6] = self.sim_data[:,self.t][6] # h
            self.sim_data[:,self.t + 1][7] = self.sim_data[:,self.t][7] # n
            self.t += 1
        
        self.sim_data[:,self.t][4] += self.action_space_x[u[0]]
        self.sim_data[:,self.t][5] += self.sim_data[:,self.t][7]*self.action_space_y[u[1]]
        self.sim_data[:,self.t][7] = -1*self.sim_data[:,self.t][7]
        
        ## modifying state that is returned is such that the origin is u0 instead of the global origin
        ## This ensures that the state x[0] is bounded by the maximum leg size while collecting data
        processed_state = np.zeros(7)
        processed_state[0:5] = np.take(self.sim_data[:,self.t].copy(), [0, 1, 2, 3, 7]) 
        processed_state[0:2] -= self.sim_data[:,self.t][4:6] # shifting origin to u
        processed_state[5:7] = self.v_des
        
        if not self.isdone():
            self.no_steps += 1
            
        return np.round(processed_state, 2), self.compute_cost(), self.isdone()
    
    def isdone(self):
        '''
        Checks if the kinematic constraints are violated
        '''
        # Computing the hip location
        hip = self.sim_data[:,self.t][0:2].copy()
        hip[1] -= self.sim_data[:,self.t][7]*(self.b/2.0)
        tmp = np.linalg.norm(hip - self.sim_data[:,self.t][4:6])
        current_leg_length = np.linalg.norm([tmp, self.h])
        if current_leg_length > self.max_leg_length:
            return True
        else:
            return False
    
    def compute_cost(self):
        '''
        Computes cost which is distance between the hip(closest hip depending on which foot is on the ground)
        and the foot + velocity of the center of mass + 1 if step length not equal to zero (after taking into
        account the offset) + 100 if episode terminates (kinematics constraints are violated)
        '''
        hip = self.sim_data[:,self.t][0:2].copy()
        hip[1] += -1*self.sim_data[:,self.t][7]*(self.b/2) # -1 is to match co ordinate axis
        u = self.sim_data[:,self.t][4:6].copy()
        cost = self.w[0]*(abs(hip - u)[0]) + self.w[0]*(abs(hip - u)[1])
        if self.isdone():
            cost += 100
        cost += self.w[1]*(abs(self.sim_data[:,self.t][2] - self.v_des[0]) \
                               + abs(self.sim_data[:,self.t][3] - self.v_des[1]))
        
        cost += self.w[2]*(abs(np.round(self.sim_data[:,self.t][4] - self.sim_data[:,self.t - 5][4], 2)))
        cost += abs(self.w[2]*(abs(np.round(self.sim_data[:,self.t][5] - self.sim_data[:,self.t - 5][5], 2)) - self.b))
          
        return cost
    
    def random_action(self):
        '''
        Genarates random action
        '''
        action_x = np.random.randint(len(self.action_space_x))
        action_y = np.random.randint(len(self.action_space_y))
        
        return np.array([action_x, action_y])

    def show_episode(self, freq):
        '''
        Shows animation
        Input :
            freq : frame rate
        '''
        sim_data = self.sim_data[:,::freq]
        
        fig = plt.figure()
        ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1))
        text_str = "top view of 2d LIPM"
        base, = ax.plot([], [], lw=3, color = 'blue')
        leg, = ax.plot([], [], lw=3, color = 'pink')
        com, = ax.plot([], [], 'o', color='red')
        foot, = ax.plot([], [], 'o', color='green')

        
        def init():
            base.set_data([], [])
            leg.set_data([], [])
            com.set_data([], [])
            foot.set_data([], [])
            return base, leg, com, foot
        
        def animate(i):
            x_com = sim_data[:,i][0]
            y_com = sim_data[:,i][1]
            u_x = sim_data[:,i][4]
            u_y = sim_data[:,i][5]
            n = sim_data[:,i][7]
            
            base.set_data([x_com, x_com], [y_com - self.b/2, y_com + self.b/2])
            leg.set_data([x_com, u_x], [y_com - n*(self.b/2), u_y])
            com.set_data([x_com], [y_com])
            foot.set_data([u_x], [u_y])
            return base,leg, com, foot
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)
        plt.grid()
        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))

In [5]:
show = False
if show:
    no_steps = 10 ## simulates 10 steps taken by the inverted pendulum
    step_time = 0.1 ## duration after which step is taken by pendulum

    env = TwoDLipmEnv(0.2, 0.13, 0.22, [0, 0, 1], [21,12])
    state = env.reset_env([0.0,0.0, 1., 0.], [0.5, 0], no_steps*step_time)
    for t in range(no_steps):
#         action = env.random_action()
#         uncomment for walking motion with v = [1, 0]
        if np.power(-1, t) > 0:
            action = [5, 11]
        elif np.power(-1, t) < 0:
            action = [5, 11]
                
        next_state, cost, done = env.step_env(action, step_time)
        print(state, action, cost, next_state, done)
        state = next_state

    env.show_episode(5)

[0.  0.  1.  0.  1.  0.5 0. ] [5, 11] 100.33 [ 2.45  0.72 16.4   7.45 -1.    0.5   0.  ] True
[ 2.45  0.72 16.4   7.45 -1.    0.5   0.  ] [5, 11] 100.33 [7.8590e+01 2.9520e+01 5.4962e+02 2.0433e+02 1.0000e+00 5.0000e-01
 0.0000e+00] True
[7.8590e+01 2.9520e+01 5.4962e+02 2.0433e+02 1.0000e+00 5.0000e-01
 0.0000e+00] [5, 11] 100.33 [ 2.573830e+03  9.613700e+02  1.802522e+04  6.735380e+03 -1.000000e+00
  5.000000e-01  0.000000e+00] True
[ 2.573830e+03  9.613700e+02  1.802522e+04  6.735380e+03 -1.000000e+00
  5.000000e-01  0.000000e+00] [5, 11] 100.33 [8.4348330e+04 3.1511980e+04 5.9073871e+05 2.2069397e+05 1.0000000e+00
 5.0000000e-01 0.0000000e+00] True
[8.4348330e+04 3.1511980e+04 5.9073871e+05 2.2069397e+05 1.0000000e+00
 5.0000000e-01 0.0000000e+00] [5, 11] 100.33 [ 2.76427630e+06  1.03270999e+06  1.93598032e+07  7.23265959e+06
 -1.00000000e+00  5.00000000e-01  0.00000000e+00] True
[ 2.76427630e+06  1.03270999e+06  1.93598032e+07  7.23265959e+06
 -1.00000000e+00  5.00000000e-01  0.00

Your browser does not support the video tag.

In [36]:
env = TwoDLipmEnv(0.2, 0.13, 0.22, [0, 1, 0], [11,9])
print(env.action_space_y)
env1 = TwoDLipmEnv(0.2, 0.13, 0.22, [0, 1, 0], [21,12])
print(env1.action_space_y)


[0.   0.04 0.09 0.13 0.16 0.19 0.24 0.29 0.35]
[0.   0.03 0.06 0.1  0.13 0.15 0.17 0.2  0.23 0.26 0.3  0.35]
